In [45]:
from glob import glob
import pandas as pd
import xarray as xr

import contextlib
import os
import pathlib
import re
import shutil
import time
from datetime import datetime
from typing import Union

import joblib
import numpy as np
import pandas as pd
import xarray as xr
from joblib import Parallel, delayed
from tqdm import tqdm

from pandas.testing import assert_frame_equal

from pismragis.processing import convert_netcdf_to_dataframe, ncfile2dataframe, nc_add_id
from pismragis.analysis import prepare_df, sensitivity_analysis


In [6]:
infiles = glob("../tests/data/ts_gris_g1200m_v2023_RAGIS_id_*_1980-1-1_2020-1-1.nc")
df = convert_netcdf_to_dataframe(infiles, add_vars=False)
df.to_parquet("../tests/data/test_scalar.parquet")
df.to_csv("../tests/data/test_scalar.csv")

Converting netcdf files to pandas.DataFrame
-------------------------------------------


Processing files: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:00<00:00, 110.34it/s]


Conversion finished in 0 seconds
-------------------------------------------


In [33]:
df = convert_netcdf_to_dataframe(infiles, add_vars=False, resample="yearly")
df = df[df.time.between("1980-01-01", "1982-01-01")].reset_index(drop=True)
df.to_parquet("../tests/data/test_scalar_YM.parquet")
df.to_csv("../tests/data/test_scalar_YM.csv")


Converting netcdf files to pandas.DataFrame
-------------------------------------------


Processing files: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:01<00:00, 23.99it/s]

Conversion finished in 1 seconds
-------------------------------------------


In [13]:
infile = "../tests/data/ts_gris_g1200m_v2023_RAGIS_id_0_1980-1-1_2020-1-1.nc"

In [14]:
df = ncfile2dataframe(infile, add_vars=False)
df.to_parquet("../tests/data/test_scalar_file.parquet")
df.to_csv("../tests/data/test_scalar_file.csv")

In [15]:
df = ncfile2dataframe(infile, add_vars=True)
df.to_parquet("../tests/data/test_scalar_file_add_vars.parquet")
df.to_csv("../tests/data/test_scalar_file_add_vars.csv")

In [39]:
n_jobs = 1

In [63]:
    X_df = (
        pd.read_parquet("../tests/data/test_scalar_YM.parquet")
        .drop(columns=["Year", "resolution_m"])
        .sort_values(by=["time", "id"])
    )
    ensemble_file = "../tests/data/gris_ragis_ocean_simple_lhs_50_w_posterior.csv"
    Y_df = sensitivity_analysis(
        X_df, ensemble_file=ensemble_file, n_jobs=n_jobs, seed=seed
    )
    Y_df.to_parquet("../tests/data/test_sensitivity.parquet")


-------------------------------------------

Program finished in 4 seconds


In [64]:
    Y_true_df = pd.read_parquet("../tests/data/test_sensitivity.parquet")
    Y_df = sensitivity_analysis(
        X_df, ensemble_file=ensemble_file, n_jobs=n_jobs, seed=seed
    )
    assert_frame_equal(Y_df, Y_true_df, rtol=1e-04)


-------------------------------------------

Program finished in 4 seconds


In [71]:
    ds = xr.open_mfdataset(
        "../tests/data/ts_gris_g1200m_v2023_RAGIS_id_*.nc",
        combine="nested",
        concat_dim="id",
        preprocess=nc_add_id,
        parallel=True,
    )
    ens = (
        ds.sel(time=slice("1980-01-01", "1982-01-01"))[ens_vars_dict.keys()]
        .resample(time="1AS")
        .mean()
    )
    X_xr_df = (
        ens.to_dataframe()
        .rename(columns=ens_vars_dict)
        .reset_index()
        .dropna()
        .sort_values(by=["time", "id"])
        .reset_index(drop=True)
    )

    Y_xr_df = sensitivity_analysis(
        X_xr_df, ensemble_file=ensemble_file, n_jobs=n_jobs, seed=seed
    )
    assert_frame_equal(Y_xr_df, Y_true_df, atol=1e-01, rtol=1e-06)


-------------------------------------------

Program finished in 4 seconds


In [62]:
assert_frame_equal(X_df, X_xr_df)

AssertionError: DataFrame.iloc[:, 2] (column name="grounding_line_flux (Gt year-1)") are different

DataFrame.iloc[:, 2] (column name="grounding_line_flux (Gt year-1)") values are different (33.33333 %)
[index]: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71]
[left]:  [-495.44469421231486, -621.2099698962929, -506.2794002183069, -405.9281420559522, -410.0130248316154, -487.3418805882292, -260.3822451538001, -609.4455916017278, -364.88419849142537, -570.408210953245, -648.0276372156358, -514.6736017760769, -618.9941917023837, -430.99957952693103, -487.41190848258526, -596.016570582292, -362.3516911117341, -296.81436772054667, -619.5575682891493, -431.4708764896751, -360.10085458034723, -579.7777990543784, -569.3830269464477, -295.06630388144004, -471.423994465

In [58]:
Y_df["vcm"] - Y_true_df["vcm"]

0     0.000000
1     0.000000
2     0.000000
3     0.000000
4     0.000000
5     0.000000
6     0.000000
7     0.000000
8     0.000000
9     0.000000
10    0.000000
11    0.000000
12    0.000000
13    0.000000
14    0.000000
15    0.000000
16    0.004471
17    0.008890
18   -0.010666
19    0.006321
20   -0.010479
21   -0.004852
22   -0.005918
23   -0.002317
Name: vcm, dtype: float64

In [ ]:
n_jobs=4
Y_true = sensitivity_analysis(df, ensemble_file=ensemble_file, n_jobs=n_jobs)[
            sens_vars
        ]

In [ ]:
Y_true.to_parquet("tests/data/test_sensitivity.parquet")

In [ ]:
ds = xr.open_mfdataset("tests/data/ts_gris_g1200m_v2023_RAGIS_id_*.nc", combine="nested", concat_dim="id", parallel=True)

In [ ]:
ens_vars = ["grounding_line_flux", "limnsw"]
ens = ds.sel(time=slice("1980-01-01" ,"1983-01-01"))[ens_vars].resample(time="1AS").mean()
ens_df = ens.to_dataframe().reset_index().dropna()

In [ ]:
ens_vars

In [ ]:
n_jobs=4

Y_true_xr = sensitivity_analysis(ens_df, ensemble_file=ensemble_file, n_jobs=n_jobs, calc_variables=ens_vars)[sens_vars]

In [ ]:
ens_df

In [ ]:
Y_true.mean()

In [ ]:
Y_true_xr.mean()

In [ ]:
Y_true

In [ ]:
import numpy as np

for i in [0, 1, 2]:
    sa_df = sensitivity_analysis(df, ensemble_file=ensemble_file, n_jobs=n_jobs)
    print(sa_df.head())
#sa_xr_df = sensitivity_analysis(ens_df, ensemble_file=ensemble_file, n_jobs=n_jobs, calc_variables=ens_vars)

In [ ]:
sa_df.head()

In [ ]:
sa_xr_df.head()

In [ ]:
import pathlib
import time
from typing import Union

import numpy as np
import pandas as pd
from joblib import Parallel, delayed
from pandas.api.types import is_string_dtype
from SALib.analyze import delta
from scipy.interpolate import interp1d
from tqdm import tqdm

from pismragis.processing import tqdm_joblib

np.random.seed(42)


In [ ]:
    df = X_df[X_df.time.between("1980-01-01", "1983-01-01")]
    id_df = (pd.read_csv(ensemble_file) * 1).replace(np.nan, 0)

    param_names = id_df.drop(columns="id").columns.values.tolist()
    for k, col in id_df.items():
        if is_string_dtype(col):
            u = col.unique()
            u.sort()
            v = [k for k, v in enumerate(u)]
            col.replace(to_replace=dict(zip(u, v)), inplace=True)
    # Define a salib "problem"
    problem = {
        "num_vars": len(id_df.drop(columns="id").columns.values),
        "names": param_names,  # Parameter names
        "bounds": zip(
            id_df.drop(columns="id").min().values,
            id_df.drop(columns="id").max().values,
        ),  # Parameter bounds
    }

    df = pd.merge(id_df, df, on="id")
    # filter out dates with only 1 experiment, e.g., due to
    # CDO changing the mid-point time when running averaging
    df = pd.concat([x for _, x in df.groupby(by="time") if len(x) > 1])
    n_dates = len(df["time"].unique())


In [ ]:
for m_date, s_df in df.groupby(by="time"):

In [ ]:
    missing_ids = list(set(id_df["id"]).difference(s_df["id"]))
    if missing_ids:
        if verbose:
            print(f"The following simulation ids are missing:\n   {missing_ids}")

        id_df_missing_removed = id_df[~id_df["id"].isin(missing_ids)]
        params = np.array(
            id_df_missing_removed.drop(columns="id").values, dtype=np.float32
        )
    else:
        params = np.array(id_df.drop(columns="id").values, dtype=np.float32)
        Si = delta.analyze(
            problem,
            params,
            response_matrix,
            num_resamples=100,
            print_to_console=False,
        )
        Si_df = Si.to_df()


In [ ]:
from glob import glob
import pandas as pd
import xarray as xr
from pismragis.processing import convert_netcdf_to_dataframe, ncfile2dataframe
from pismragis.analysis import prepare_df, sensitivity_analysis

import pathlib
import time
from typing import Union

import numpy as np
import pandas as pd
from joblib import Parallel, delayed
from pandas.api.types import is_string_dtype
from SALib.analyze import delta
from scipy.interpolate import interp1d
from tqdm import tqdm

from pismragis.processing import tqdm_joblib


In [ ]:
cd ..

In [ ]:
X_df = pd.read_parquet("tests/data/test_scalar_YM.parquet")

In [ ]:
    sens_vars = [
        "vcm",
        "gamma_T",
        "thickness_calving_threshold",
        "ocean_file",
        "sia_e",
        "ssa_n",
        "pseudo_plastic_q",
        "till_effective_fraction_overburden",
        "phi_min",
        "phi_max",
        "z_min",
        "z_max",
    ]
    sensitivity_indices: Union[str, list] = ["delta", "S1"]
    calc_variables = ["grounding_line_flux (Gt year-1)", 
                      "limnsw (kg)"]
    ensemble_file = "tests/data/gris_ragis_ocean_simple_lhs_50_w_posterior.csv"
    ens_df = X_df

In [ ]:
    """
    Calculate sensitivity indices using SALIB and return pd.DataFrame.

    Parameters
    ----------
    df : pd.DataFrame
        A DataFrame procduced with processing.convert_netcdf_to_dataframe
    ensemble_file: str
        A csv file that maps ensemble member id's to parameters
    calc_variables: list
        A list of variables for which sensitivity indices are calculated
    n_jobs: int
        Number of parallel workers
    sensitivity_indices: str or list
        A list of sensitivity indices

    Returns
    -------
    pd.DataFrame
        A Pandas DataFrame with sensitivity indices
    """

    if isinstance(calc_variables, str):
        calc_variables = [calc_variables]

    if isinstance(sensitivity_indices, str):
        sensitivity_indices = [sensitivity_indices]

        print("Running sensitivity analysis")
    print("-------------------------------------------\n")
    start_time = time.perf_counter()

    # remove True/False
    id_df = (pd.read_csv(ensemble_file) * 1).replace(np.nan, 0)

    param_names = id_df.drop(columns="id").columns.values.tolist()
    for k, col in id_df.items():
        if is_string_dtype(col):
            u = col.unique()
            u.sort()
            v = [k for k, v in enumerate(u)]
            col.replace(to_replace=dict(zip(u, v)), inplace=True)
    # Define a salib "problem"
    problem = {
        "num_vars": len(id_df.drop(columns="id").columns.values),
        "names": param_names,  # Parameter names
        "bounds": zip(
            id_df.drop(columns="id").min().values,
            id_df.drop(columns="id").max().values,
        ),  # Parameter bounds
    }

    df = pd.merge(id_df, ens_df, on="id")
    # filter out dates with only 1 experiment, e.g., due to
    # CDO changing the mid-point time when running averaging
    df = pd.concat([x for _, x in df.groupby(by="time") if len(x) > 1])
    n_dates = len(df["time"].unique())


In [ ]:
    verbose = False
    missing_ids = list(set(id_df["id"]).difference(s_df["id"]))
    if missing_ids:
        if verbose:
            print(f"The following simulation ids are missing:\n   {missing_ids}")

        id_df_missing_removed = id_df[~id_df["id"].isin(missing_ids)]
        params = np.array(
            id_df_missing_removed.drop(columns="id").values, dtype=np.float32
        )
    else:
        params = np.array(id_df.drop(columns="id").values, dtype=np.float32)


In [ ]:
dates = []
s_dfs = []
for m_date, s_df in df.groupby(by="time"):
    dates.append(m_date)
    s_dfs.append(s_df)

In [ ]:
m_date = dates[0]
s_df = s_dfs[0]

In [ ]:
seed = 42
for i in range(4):
    for calc_variable in calc_variables:
        response_matrix = s_df[calc_variable].values
        Si = delta.analyze(
            problem,
            params,
            response_matrix,
            num_resamples=100,
            print_to_console=False,
            seed=None
        )
        Si_df = Si.to_df()
        print(Si_df.head())


In [ ]:
delta.analyze?

In [ ]:
from SALib.test_functions import Ishigami
from SALib.analyze import delta
from SALib.sample import latin

problem = {
    'num_vars': 3,
    'names': ['x1', 'x2', 'x3'],
    'bounds': [[-3.14159265359, 3.14159265359],
               [-3.14159265359, 3.14159265359],
               [-3.14159265359, 3.14159265359]]
}

X = latin.sample(problem, 1000)
Y = Ishigami.evaluate(X)
for _ in range(4):
    Si = delta.analyze(problem, X, Y, print_to_console=True, seed=42)
    print(Si.to_df().head())

In [ ]:
import SALib

In [ ]:
    seed = 42

    sens_vars = [
        "vcm",
        "gamma_T",
        "thickness_calving_threshold",
        "ocean_file",
        "sia_e",
        "ssa_n",
        "pseudo_plastic_q",
        "till_effective_fraction_overburden",
        "phi_min",
        "phi_max",
        "z_min",
        "z_max",
    ]


In [ ]:
    X_df = pd.read_parquet("tests/data/test_scalar_YM.parquet")
    # Y_true = pd.read_parquet("tests/data/test_sensitivity.parquet")[sens_vars].mean()
    ensemble_file = "tests/data/gris_ragis_ocean_simple_lhs_50_w_posterior.csv"
    for n_jobs in [1]:
        Y = sensitivity_analysis(
            X_df, ensemble_file=ensemble_file, n_jobs=n_jobs, seed=seed
        )[sens_vars].mean()
        assert_array_almost_equal(Y, Y_true, decimal=True)


In [ ]:
from numpy.testing import assert_array_almost_equal
from pandas.testing import assert_frame_equal


In [ ]:
Y_true = sensitivity_analysis(
            X_df, ensemble_file=ensemble_file, n_jobs=n_jobs, seed=seed
        )
Y_true.to_parquet("tests/data/test_sensitivity.parquet")

In [ ]:
assert_frame_equal(Y, Y_true)

In [ ]:
Y_true_df = pd.read_parquet("tests/data/test_sensitivity.parquet")

In [ ]:
Y_true

In [ ]:

X_df = pd.read_parquet("tests/data/test_scalar_YM.parquet")



In [ ]:
    ds = xr.open_mfdataset(
        "tests/data/ts_gris_g1200m_v2023_RAGIS_id_*.nc",
        combine="nested",
        concat_dim="id",
        parallel=True,
    )
    ens_vars = ["grounding_line_flux", "limnsw"]
    ens = (
        ds.sel(time=slice("1980-01-01", "1983-01-01"))[ens_vars]
        .resample(time="1AS")
        .mean()
    )
    X_xr_df = ens.to_dataframe().reset_index().dropna()


In [ ]:
X_df.sort_values(by=["time", "id"])

In [ ]:
X_xr_df.sort_values(by=["time", "id"])

In [ ]:
ens_vars_dict = {"grounding_line_flux": "grounding_line_flux (Gt year-1)", "limnsw": "limnsw (kg)"}


In [ ]:
ens_vars.keys()

In [ ]:
    ds = xr.open_mfdataset(
        "tests/data/ts_gris_g1200m_v2023_RAGIS_id_*.nc",
        combine="nested",
        concat_dim="id",
        parallel=True,
    )
    ens_vars = ["grounding_line_flux", "limnsw"]
    ens = (
        ds.sel(time=slice("1980-01-01", "1983-01-01"))[ens_vars_dict.keys()]
        .resample(time="1AS")
        .mean()
    )
    X_xr_df = ens.to_dataframe().rename(columns=ens_vars_dict).reset_index().dropna()


In [ ]:
X_df

In [ ]:
    X_df = pd.read_parquet("tests/data/test_scalar_YM.parquet")
    Y_true = pd.read_parquet("tests/data/test_sensitivity.parquet")
    ensemble_file = "tests/data/gris_ragis_ocean_simple_lhs_50_w_posterior.csv"
    for n_jobs in [1, 2, 4]:
        Y = sensitivity_analysis(
            X_df, ensemble_file=ensemble_file, n_jobs=n_jobs, seed=seed
        )
        assert_frame_equal(Y, Y_true)


In [ ]:
    ds = xr.open_mfdataset(
        "tests/data/ts_gris_g1200m_v2023_RAGIS_id_*.nc",
        combine="nested",
        concat_dim="id",
        parallel=True,
    )
    ens = (
        ds.sel(time=slice("1980-01-01", "1983-01-01"))[ens_vars_dict.keys()]
        .resample(time="1AS")
        .mean()
    )
    X_xr_df = ens.to_dataframe().rename(columns=ens_vars_dict).reset_index().dropna()

    Y_true_df = pd.read_parquet("tests/data/test_sensitivity.parquet")
    ensemble_file = "tests/data/gris_ragis_ocean_simple_lhs_50_w_posterior.csv"
    for n_jobs in [1, 2, 4]:
        Y_xr_df = sensitivity_analysis(
            X_xr_df,
            ensemble_file=ensemble_file,
            n_jobs=n_jobs,
            seed=seed,
        )
        assert_frame_equal(Y_xr_df, Y_true_df)


In [ ]:
Y_xr_df["vcm"]-Y_df["vcm"]

In [ ]:
Y_df["vcm"]

In [ ]:
assert_frame_equal(Y_xr_df, Y_df)

In [ ]:
Y_true_df

In [ ]:
Y_df.to_parquet("tests/data/test_sensitivity.parquet")

In [ ]:
    ds = xr.open_mfdataset(
        "tests/data/ts_gris_g1200m_v2023_RAGIS_id_*.nc",
        combine="nested",
        concat_dim="id",
        parallel=True,
    )
    ens = (
        ds.sel(time=slice("1980-01-01", "1982-01-01"))[ens_vars_dict.keys()]
        .resample(time="1AS")
        .mean()
    )
    X_df = ens.to_dataframe().rename(columns=ens_vars_dict).reset_index().dropna()

    Y_true_df = pd.read_parquet("tests/data/test_sensitivity.parquet")
    ensemble_file = "tests/data/gris_ragis_ocean_simple_lhs_50_w_posterior.csv"
    for n_jobs in [1, 2, 4]:
        Y_df = sensitivity_analysis(
            X_df,
            ensemble_file=ensemble_file,
            n_jobs=n_jobs,
            seed=seed,
        )
        assert_frame_equal(Y_df, Y_true_df)


In [ ]:
    X_df = pd.read_parquet("tests/data/test_scalar_YM.parquet")
    Y_df_true = pd.read_parquet("tests/data/test_sensitivity.parquet")
    ensemble_file = "tests/data/gris_ragis_ocean_simple_lhs_50_w_posterior.csv"
    for n_jobs in [1, 2, 4]:
        Y_df = sensitivity_analysis(
            X_df, ensemble_file=ensemble_file, n_jobs=n_jobs, seed=seed
        )
        assert_frame_equal(Y_df, Y_true_df)


In [ ]:
Y_df

In [ ]:
Y_df

In [ ]:
    ds = xr.open_mfdataset(
        "tests/data/ts_gris_g1200m_v2023_RAGIS_id_*.nc",
        combine="nested",
        concat_dim="id",
        parallel=True,
    )
    ens = (
        ds.sel(time=slice("1980-01-01", "1983-01-01"))[ens_vars_dict.keys()]
        .resample(time="1AS")
        .mean()
    )
    X_xr_df = ens.to_dataframe().rename(columns=ens_vars_dict).reset_index().dropna()

    Y_true_df = pd.read_parquet("tests/data/test_sensitivity.parquet")
    ensemble_file = "tests/data/gris_ragis_ocean_simple_lhs_50_w_posterior.csv"
    for n_jobs in [1, 2, 4]:
        Y_xr_df = sensitivity_analysis(
            X_xr_df,
            ensemble_file=ensemble_file,
            n_jobs=n_jobs,
            seed=seed,
        )
        assert_frame_equal(Y_xr_df, Y_true_df)


In [ ]:
    X_df = pd.read_parquet("tests/data/test_scalar_YM.parquet")
    Y_true_df = pd.read_parquet("tests/data/test_sensitivity.parquet")
    ensemble_file = "tests/data/gris_ragis_ocean_simple_lhs_50_w_posterior.csv"
    for n_jobs in [1, 2, 4]:
        Y_df = sensitivity_analysis(
            X_df, ensemble_file=ensemble_file, n_jobs=n_jobs, seed=seed
        )
        assert_frame_equal(Y_df, Y_true_df)


In [ ]:
X_df.sort_values(by=["time", "id"])

In [ ]:
X_xr_df.sort_values(by=["time", "id"])

In [ ]:
    X_df = (
        pd.read_parquet("tests/data/test_scalar_YM.parquet")
        .drop(columns=["Year", "resolution_m"])
        .sort_values(by=["time", "id"])
        .reset_index(drop=True)
    )


In [ ]:
    ds = xr.open_mfdataset(
        "tests/data/ts_gris_g1200m_v2023_RAGIS_id_*.nc",
        combine="nested",
        concat_dim="exp",
        parallel=True,
    )
    ens = (
        ds.sel(time=slice("1980-01-01", "1983-01-01"))[ens_vars_dict.keys()]
        .resample(time="1AS")
        .mean()
    )
    X_xr_df = (
        ens.to_dataframe()
        .rename(columns=ens_vars_dict)
        .sort_values(by=["time", "id"])
        .reset_index()
    )


In [ ]:
assert_frame_equal(X_df, X_xr_df)

In [ ]:
ds["exp"]

In [ ]:
X_xr_df

In [ ]:
ens.to_dataframe().reset_index()

In [ ]:
X_df["id"].unique()

In [ ]:
xr.open_mfdataset?

In [ ]:
ds.encoding["source"]

In [ ]:
import re

In [ ]:

ds = xr.open_mfdataset(
    "tests/data/ts_gris_g1200m_v2023_RAGIS_id_*.nc",
    combine="nested",
    concat_dim="id",
    parallel=True,
    preprocess=preprocess
    )


In [ ]:
def preprocess(ds):
    m_id_re = re.search("id_(.+?)_", ds.encoding["source"])
    ds.expand_dims("id")
    assert m_id_re is not None
    m_id: Union[str, int]
    try:
        m_id = int(m_id_re.group(1))
    except:
        m_id = str(m_id_re.group(1))    
    ds["id"] = m_id
    return ds

In [ ]:

ds["id"]

In [ ]:
m_id_re

In [ ]:
m_id

In [ ]:
cd ..

In [ ]:
X_xr_df.mean()

In [ ]:
    X_df = (
        pd.read_parquet("tests/data/test_scalar_YM.parquet")
        .drop(columns=["Year", "resolution_m"])
        .sort_values(by=["time", "id"])
        .reset_index(drop=True)
    )


In [ ]:
X_df.mean()

In [ ]:
    ds = xr.open_mfdataset(
        "tests/data/ts_gris_g1200m_v2023_RAGIS_id_*.nc",
        combine="nested",
        concat_dim="id",
        preprocess=nc_add_id,
        parallel=True,
    )
    ens = (
        ds.sel(time=slice("1980-01-01", "1982-01-01"))[ens_vars_dict.keys()]
        .resample(time="1YS")
        .mean()
    )
    X_xr_df = (
        ens.to_dataframe()
        .rename(columns=ens_vars_dict)
        .reset_index()
        .dropna()
        .sort_values(by=["time", "id"])
        .reset_index(drop=True)
    )

In [ ]:
ens_vars_dict = {
    "grounding_line_flux": "grounding_line_flux (Gt year-1)",
    "limnsw": "limnsw (kg)",
}


In [ ]:
assert_frame_equal(X_df, X_xr_df)

In [ ]:
X_df


In [ ]:
X_xr_df

In [ ]:
X_df["grounding_line_flux (Gt year-1)"]

In [72]:
def test_sensitivity_analysis_from_xarray():
    ds = xr.open_mfdataset(
        "tests/data/ts_gris_g1200m_v2023_RAGIS_id_*.nc",
        combine="nested",
        concat_dim="id",
        preprocess=nc_add_id,
        parallel=True,
    )
    ens = (
        ds.sel(time=slice("1980-01-01", "1982-01-01"))[ens_vars_dict.keys()]
        .resample(time="1AS")
        .mean()
    )
    X_df = (
        ens.to_dataframe()
        .rename(columns=ens_vars_dict)
        .reset_index()
        .dropna()
        .sort_values(by=["time", "id"])
        .reset_index(drop=True)
    )

    Y_true_df = pd.read_parquet("tests/data/test_sensitivity.parquet")
    ensemble_file = "tests/data/gris_ragis_ocean_simple_lhs_50_w_posterior.csv"
    for n_jobs in [1, 2]:
        Y_df = sensitivity_analysis(
            X_df,
            ensemble_file=ensemble_file,
            n_jobs=n_jobs,
            seed=seed,
        )
        assert_frame_equal(Y_df, Y_true_df, atol=1e-1, rtol=1e-6)


In [78]:

test_sensitivity_analysis_from_xarray()

-------------------------------------------

Program finished in 4 seconds
-------------------------------------------



Processing date: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:03<00:00,  1.03s/it]

Program finished in 3 seconds


In [77]:
cd ..

/Users/andy/base/pism-ragis
